### Adversarial AE

#### Adversarial ae라고 naming 하기는 하였으나, 기존에 나와있는 vae를 이용한 구조는 아니고,
#### 정민정님이 사용한것과 같은 형태의 AE

In [2]:
import pandas as pd
import numpy as np
import pyreadr as py # library to read .Rdata files in python
import os
import tensorflow as tf
from time import time
import pickle
import datetime
import random
import math
import csv
import time
import sys

In [3]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,KFold
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,f1_score,log_loss,recall_score,classification_report
from sklearn.preprocessing import scale, robust_scale, minmax_scale, maxabs_scale

In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset  
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [5]:
from src.models.model import AE_base
from src.data.common.dataset import FontDataset, PickledImageProvider

In [6]:
import torch
from torch.nn import functional as F
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [7]:
batch_size = 8
validation_split = .15
test_split = .05
shuffle_dataset = True
random_seed = 42

lr = 0.0001

log_interval = 10
epochs = 200

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
from src.data.common.dataset import FontDataset, PickledImageProvider

In [9]:
# get Dataset
data_dir = 'src/data/dataset/allfonts/'
train_set = FontDataset(PickledImageProvider(data_dir+'train.obj'))
valid_set = FontDataset(PickledImageProvider(data_dir+'val.obj'))
test_set = FontDataset(PickledImageProvider(data_dir+'test.obj'))

processed 1000 examples
processed 2000 examples
processed 3000 examples
processed 4000 examples
processed 5000 examples
processed 6000 examples
processed 7000 examples
processed 8000 examples
processed 9000 examples
processed 10000 examples
processed 11000 examples
processed 12000 examples
processed 13000 examples
processed 14000 examples
processed 15000 examples
processed 16000 examples
processed 17000 examples
processed 18000 examples
processed 19000 examples
processed 20000 examples
processed 21000 examples
processed 22000 examples
processed 23000 examples
processed 24000 examples
processed 25000 examples
processed 26000 examples
processed 27000 examples
processed 28000 examples
processed 29000 examples
processed 30000 examples
processed 31000 examples
processed 32000 examples
processed 33000 examples
processed 34000 examples
processed 35000 examples
processed 36000 examples
processed 37000 examples
processed 38000 examples
processed 39000 examples
processed 40000 examples
processed

In [10]:
# get idx samplers
train_set_size = len(train_set)
valid_set_size = len(valid_set)
train_idxs = list(range(train_set_size))
valid_idxs = list(range(valid_set_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(train_idxs)
    np.random.shuffle(valid_idxs)

train_sampler = SubsetRandomSampler(train_idxs)
valid_sampler = SubsetRandomSampler(valid_idxs)

# get data_loaders
train_loader = DataLoader(train_set, 
                      batch_size=batch_size,
                      sampler=train_sampler
                      )
valid_loader = DataLoader(valid_set,
                        batch_size=batch_size,
                        sampler=valid_sampler
                        )
test_loader = DataLoader(test_set,
                        batch_size=len(test_set)
                        )

---

In [11]:
input_category_size=5; input_alpha_size=52; input_font_size=128*128

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()     

        self.model = nn.Sequential(
            nn.Linear(input_font_size, 8192),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(8192, 4096),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(4096, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )        
    
    def forward(self, x_font):
        x_font = x_font.view(x_font.shape[0], -1)
        out = self.model(x_font)
        
        return out.squeeze()

---
### generator

In [12]:
from src.models.layers import Encoder_base, Decoder_base
#from src.models.layers import Encoder_category, Decoder_category

In [13]:
## 훈철 코드 
input_category_size=5; input_alpha_size=52; input_font_size=128*128; input_noise_size = 16;
output_font_size=128*128

class AE_base(nn.Module):
    def __init__(self, category_size=5, alpha_size=52, font_size=128*128,z_size=64):
        super(AE_base, self).__init__()
        self.Encoder = Encoder_base(input_category_size=category_size,
                                    input_alpha_size=alpha_size,
                                    input_font_size=font_size,
                                    z_size=z_size)
        self.Decoder = Decoder_base(z_latent_size=z_size,
                                    z_category_size=category_size,
                                    z_alpha_size=alpha_size,
                                    output_font_size=font_size)
    
    def forward(self, x_font, alpha_vector, category_vector=None):
        
        origin_shape = x_font.shape
        x_font = x_font.view(x_font.shape[0], -1)
        alpha_vector = alpha_vector.view(alpha_vector.shape[0], -1)
        
        if category_vector is not None:
            category_vector = category_vector.view(category_vector.shape[0], -1)
        
        if category_vector is not None:
            x = torch.cat([x_font, category_vector, alpha_vector], dim=1)
        else:
            x = torch.cat([x_font, alpha_vector], dim=1)
        
        z_latent = self.Encoder(x)
        
        #z_latent = z_latent.view(z_latent.shape[0], -1)
        if category_vector is not None:    
            z = torch.cat([z_latent, category_vector, alpha_vector], dim=1)
        else:
            z = torch.cat([z_latent, alpha_vector], dim=1)
        
        x_hat = self.Decoder(z)
        x_hat = x_hat.view(origin_shape)
        
        return x_hat


In [14]:
discriminator = Discriminator().cuda()
#generator = Generator().cuda()
generator = AE_base().cuda()

In [15]:
bce_criterion = nn.BCELoss().cuda()
mse_criterion = nn.MSELoss().cuda()

d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

In [16]:
def train_discriminator(batch_size, generator, discriminator, images, alpha_vector, category_vector):
    d_optimizer.zero_grad()
    
    #train with the real image (discriminator)
    outputs = discriminator(images)
    real_loss = bce_criterion(outputs, Variable(torch.ones(batch_size)).cuda()) 
    real_score = outputs 
        
    #train with the real image (generator)
    fake_images = generator(images, alpha_vector, category_vector) 
    outputs = discriminator(fake_images)
    fake_loss = bce_criterion(outputs, Variable(torch.zeros(batch_size)).cuda()) 
    fake_score = outputs 

    d_loss = real_loss + fake_loss 
    d_loss.backward() 
    d_optimizer.step()
    
    return d_loss, real_score, fake_score



In [17]:
def train_generator(batch_size, alpha_vector, category_vector, images, generator, discriminator):
    
    g_optimizer.zero_grad()
   
    #get output from generator and discriminator   
    fake_images = generator(images, alpha_vector, category_vector) 
    outputs = discriminator(fake_images)
        
    g_mse_loss= mse_criterion(fake_images, images)
    g_bce_loss = bce_criterion(outputs, Variable(torch.ones(batch_size)).cuda())
    
    g_loss = g_mse_loss + g_bce_loss
    g_loss.backward()
    g_optimizer.step()
    
    return g_loss



In [18]:
# set number of epochs and initialize figure counter
num_epochs = 100
num_batches = len(train_loader)
num_fig = 0


for epoch in range(num_epochs):
    for n, (vectors, font, _) in enumerate(train_loader):
                
        alpha_vector = vectors['alphabet_vector']
        category_vector = vectors['category_vector']
        
        images, alpha_vector = font.float().to(device), alpha_vector.float().to(device)
        category_vector = category_vector.float().to(device)
 
        temp_batch_size = alpha_vector.size()[0]
    
        # Train the discriminator
        d_loss, real_score, fake_score =train_discriminator(temp_batch_size, generator, discriminator, images, alpha_vector, category_vector)
       
        
        # Train the generator
        g_loss = train_generator(temp_batch_size, alpha_vector, category_vector, images, generator, discriminator)
     
        if n % 1000 == 0: print(n, " / ")
            
        
    if epoch % 1 == 0:
        print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, g_loss: %.4f, ' 
              'D(x): %.2f, D(G(z)): %.2f' 
              %(epoch + 1, num_epochs, n+1, num_batches, d_loss.item(), g_loss.item(),
                real_score.data.mean(), fake_score.data.mean()))


0  / 


KeyboardInterrupt: 

In [ ]:
# save
#savePath = "D:/투빅스 2019/discriminator_1214.pth"
#torch.save(discriminator.state_dict(), savePath)



In [ ]:
# load
#new_model = TestModel()
#new_model.load_state_dict(torch.load("./output/test_model.pth"))


---

In [ ]:
z = Variable(torch.randn(52, 16)).cuda()
fake_alpha_vector =  Variable(torch.FloatTensor(np.eye(52))).cuda()
fake_category_vector =  Variable(torch.FloatTensor(np.eye(5)[np.random.choice(5,52)])).cuda()


In [ ]:
images = generator(z, fake_alpha_vector, fake_category_vector).unsqueeze(1)

In [ ]:
grid = make_grid(images, nrow=52, normalize=True)

In [ ]:
images = images.view(images.size(0), 128, 128)

In [ ]:
plt.imshow(images[3].cpu().detach().numpy())

In [ ]:
plt.figure(figsize=(10, 200))
for i in range(52):
    plt.subplot(50, 2, i+1)
    plt.imshow(images[i].cpu().detach().numpy())